# Case Study: Walmart Sales Forcast
## With SQL, and Power BI

### Table of Contents
1. [Project Summary](#projectsummary)
    * 1.1 [Background](#projectbackground)
    * 1.2 [Dataset](#dataset)
2. [Objectives](#objectives)
    * 2.1 [Main Objective](#mainobjective)
3. [Preparing the data](#prepare)
    * 3.1 [Big Query](#bigquery)
4. [Processing the data](#process)
    * 4.1 [Datasets Used](#datasetsused)
5. [Analyzing the Data](#analyze)
    * 5.1 [Query 1: Size of Store and Sale](#query1)
    * 5.2 [Query 2: Which Date had the most sales](#query2)
    * 5.3 [Query 3: Sales leading up to holidays](#query3)
6. [Machine Learning Predictions](#ML)
    * 6.1 [Data Exploration and Merge](#datamergeexplore)

## 1. Project Summary <a class="anchor" id="projectsummary"></a>

#### 1.1 Project Background <a class="anchor" id="projectbackground"></a>
Inspiration for this project came from [**Help Walmart Predict Manage Inventory**](https://www.kaggle.com/competitions/dsedelhi) contest held by Kaggle. This contest was shut on 30/May/2019 but I thought it would serve as an excellent opportunity to improve my skills in SQL and Power BI. The premise for this contest was to use historical sales data to forecast future sales of products. Sale forecasting has a strong influence on the performance of a company's success, therefore proper analysis of this data is crucial for the company's performance in the coming year. The accurate forecast will lead to better business decisions and help the retailer to take the necessary steps and measures to plan their yearly budgets and investments. 

For this Case Study I decided to ask the following questions:
* Whether the size of the store had an affect on sales?
* Which days had the highest sales?
* Conclude whether the weather has an essential impact on sales.
* Do the sales always rise near the holiday season for all the years?



#### 1.2 Dataset <a class="anchor" id="dataset"></a>
As the contest has been shut for 4 years, the dataset offered by Kaggle was no longer available to the public. After searching for a similar dataset I decide on using the  [**Walmart Recruiting Store Sales Forecasting**](https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting) dataset made available through Kaggle. This dataset contains four separate CSV files named features, stores, test, and train. All four csv files can be linked by the first column labelled 'Store'. Also, this dataset focuses on data collected between 2010-02-05 to 2013-12-30. The key dates in between that time frame are as follows:
* Super Bowl: 07-Feb-10, 06-Feb-11, 05-Feb-12, 03-Feb-13
* Labor Day: 06-Sep-10, 05-Sep-11, 07-Sep-12, 02-Sep-13
* Thanksgiving: 25-Nov-10, 24-Nov-11, 22-Nov-12, 28-Nov-13
* Christmas: 25-Dec-10, 25-Dec-11, 25-Dec-12, 25-Dec-13

## 2. Objectives <a class="anchor" id="objectives"></a>​
#### 2.1 Main Objective <a class="anchor" id="mainobjective"></a>
The main objective of this case study is to predict future sales for Walmart retail stores. As the datasets give us access to information such as weather, whether or not the sales day was a holiday, the price of fuel, and unemployment rates, we will also examine if any of these affect sales.

## 3. Prepare Section <a class="anchor" id="prepare"></a>
#### 3.1 BigQuery <a class="anchor" id="bigquery"></a>
I decided to use BigQuery as a platform to perform all SQL actions as cloud-based data warehouses such as BigQuery and Azure are in high demand. BigQuery is Google's fully managed, serverless data warehouse, it allows for quick analysis of a substantial amount of data. BigQuery also supports a standard version of SQL (ANSI SQL), another feature that makes BigQuery a great tool for this case study.

## 4. Processing the Data <a class="anchor" id="process"></a>
#### 4.1 Datasets Used <a class="anchor" id="datasetsused"></a>
As mentioned earlier the datasource contains 4 datasets:
​
* Features - containd: Store, Date, Temperature, Fuel Price, Markdown's 1 to 5, CPI, Unemployment, and IsHoliday columns
* Stores - contained: Store, Type, and Size columns
* Test - contained: Store, Dept, Date, and IsHoliday columns
* Train - contained: Store, Dept, Date, Weekly_Sales, and IsHoliday columns

## 5. Analyzing the Data <a class="anchor" id="analyze"></a>
#### 5.1 Query 1: Size of Store and Sales <a class="anchor" id="query1"></a>
The first SQL query I performed was to group the Stores by average weekly sales, type, and size. This was done to see if the size or type of the stores had an overall effect on how that store performed during the period of data collection. An Inner Join was used as we needed information from the Train and Stores tables. I also used the AVG function to average the weekly sales and label the column as AVG_Sales. Final the new table was grouped by Store, Size, and Type, then put in descending order based on the average weekly sales. 

In [ ]:
''''
SELECT a.Store,
  AVG(a.Weekly_Sales) AS Avg_Sales,  
  b.Size,  
  b.Type  
FROM `walmart-forecast-case-study.forecast_data.train` AS a
INNER JOIN `walmart-forecast-case-study.forecast_data.stores` AS b ON a.Store = b.Store
GROUP BY a.Store, b.Size, b.Type
ORDER BY Avg_Sales
'''

#### 5.2 Query 2: Which Date had the most sales <a class="anchor" id="query2"></a>
For the next query I used the MAX function to sort the data by highest weekly sales in a descending order. Bringing the highest grossing days to the top of my query results. I also decided to add the IsHoliday column to my query as this would help indicitate if sale rose during holiday dates. From my results I found that holiday dates and days surrounding the holidays had the high sales volume. The Christmas period and the day following Thanksgiving (Black Friday) saw some of the highest sales in this time-frame.

In [ ]:
''''
SELECT Date,
  MAX(Weekly_Sales) AS Max_sales,
  IsHoliday
FROM `walmart-forecast-case-study.forecast_data.train`
GROUP BY Date, IsHoliday
ORDER BY Max_sales desc
'''

#### 5.3 Query 3: Sales leading up to holidays <a class="anchor" id="query3"></a>
This SQL query was used to view the average sales each store had a week before and after a Holiday. The example below was for Christmas of 2010, but similar queries were written for Thanksgiving, Labor Day, and the Super Bowl. My aim was to track customer activity during the holiday dates to see if there was a spike in sales either before or after each holiday.

In [ ]:
''''
SELECT Store,
  Date,
  AVG(Weekly_Sales) AS Avg_Sales
FROM `walmart-forecast-case-study.forecast_data.train`
WHERE Date >= '2010-12-18'
AND Date <= '2011-01-01'
GROUP BY Store, Date
'''

## 6. Machine Learning Predictions <a class="anchor" id="ML"></a>
Here I will use some Machine Learning models to make some predictions. I will use the test dataset to create a submission file that will be used for a late submission in the competition.


In [ ]:
# Imports
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Reads CSV files
train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')

#### 6.1 Data Exploration and Merge <a class="anchor" id="datamergeexplore"></a>
Below I will perform some further explorations to fully understand the data I am working with. After this step I will merge the datasets based on features i find appropriate. This new dataframe (df) will be the one used to train my models.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
stores.head()

In [ ]:
features.head()

In [ ]:
train.describe()

In [ ]:
features.describe()

In [ ]:
# Performs merge so all appropriate data is in one set df
df = train.merge(stores, on='Store')
feature_cols = ['Store', 'Date', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
df = df.merge(features[feature_cols], on=['Store', 'Date'])

df.head()

In [ ]:
# Performs merge step for test data, df_test
df_test = test.merge(stores, on='Store')
feature_cols = ['Store', 'Date', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
df_test = df_test.merge(features[feature_cols], on=['Store', 'Date'])

df_test.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
# Checks for missing data
df.isna().sum()